To save on file space in the git repository, the cell outputs below have been cleared. Just click the restart/run button to populate the below cells.

In [ ]:
import hashlib
from pyhibp import pwnedpasswords as pw
from pyhibp import set_user_agent

In [ ]:
# The User Agent must be set prior to calling any additional functions.
# Per the API: "The user agent should accurately describe the nature of the API consumer
#   such that it can be clearly identified in the request. Not doing so may result
#   in the request being blocked."
set_user_agent(ua="pyHIBP-binder/An example Jupyter Notebook showing how to use the pwnedpasswords module of pyhibp")

In [ ]:
"""
Query the HIBP API to see if a password has been breached.
"""
resp = pw.is_password_breached(password="secret")

if resp:
    print("Password breached! It appears {0} time(s) in the HIBP database.".format(resp))
else:
    # A non-matching password will be falsy (integer zero (0) is the return value)
    print("Nice! That password hasn't been breached.")

In [ ]:
"""
While a password is converted to a SHA-1 hash, then partially sent to the HIBP backend, 
we can also pass in a SHA-1 hash directly.
"""
hashed = hashlib.sha1("secret1".encode("utf-8")).hexdigest()
print("Using the SHA-1 hash: {0}".format(hashed))

# NB: We're actually only sending hashed[0:5] to the HIBP backend
resp = pw.is_password_breached(sha1_hash=hashed)

if resp:
    print("Password breached! It appears {0} times in the HIBP database.".format(resp))
else:
    print("Nice! That password hasn't been breached.")

In [ ]:
"""
Optionally, `pwnedpasswords` can be used to search hash suffixes via ``suffix_search(hash_prefix=prefix)``, by supplying
the hash prefix of a SHA-1 hash (that is, the first five hash characters).

When called, the HIBP API returns a listing of hash suffixes, along with the number of times that suffix has been seen.
The format of the returned data is in the format of ``suffix:count``; for example, for the prefix of ``42042``, returned
list items might be:

```
    005F4A4B9265A2BABE10B1A9AB9409EA3F0:1
    00D6F0319225107BD5736B72717BD381660:8
    01355DCE0B54F0E8DBBBA8F7B9A9872858A:15
    0163E1C872A64A62625F5EB2F3807B7F90B:2
    020DDE278E6A9C05B356C929F254CE6AED5:1
    021EFB4FAE348050D9EDCD10F8B6A87C957:4
```

Consequently, the HIBP API is never provided the full SHA-1 hash for a queried password, and thus remains undisclosed.
Internally, ``is_password_breached`` also uses this method so as to avoid disclosing a complete password hash outside
of the calling application.

For additional information on the k-anonymity model used here,
see < https://www.troyhunt.com/ive-just-launched-pwned-passwords-version-2/ > for more information.
"""
hashed = hashlib.sha1("secret1".encode("utf-8")).hexdigest()
print("Using the SHA-1 partial hash: {0}".format(hashed[0:5]))

# NB: We're actually only sending hashed[0:5] to the HIBP backend
resp = pw.suffix_search(hash_prefix=hashed[0:5])

# Since we're only sending the first five characters, the calling application needs to parse through the resulting
# list, and find the remainder of the hash, if present, to determine if the password in question was breached.

print("There are {0} partial SHA-1 hashes in the response which matched the leading prefix of {1}.".format(len(resp), hashed[0:5].upper()))
print()

# Since we only passed a partial hash, we need to manually look for the corresponding hash in the response.
for partial_hash in resp:
    if hashed[5:] in partial_hash.lower():
        print("The partial hash which the HIBP Pwned Passwords API returned was: {0}".format(partial_hash.split(':')[0]))
        print("It was found used {0} time(s).".format(partial_hash.split(':')[1]))
        print("Inside the ``resp`` list, it would look like the following line...")
        print()
        print(partial_hash)